### download the last month of the CDS raw forecasts 

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import pathlib
from datetime import datetime
from dateutil.relativedelta import relativedelta
import dateparser

In [4]:
from dask import delayed, compute
from dask.diagnostics import ProgressBar

In [5]:
HOME = pathlib.Path.home()

In [6]:
CWD = pathlib.Path.cwd()

In [7]:
CWD

PosixPath('/home/nicolasf/operational/ICU/development/hotspots/code/ICU_Water_Watch/notebooks/C3S')

### import local functions for the downloading of the C3S forecasts 

In [8]:
sys.path.append('../..')

In [9]:
from ICU_Water_Watch import C3S, domains

### show the predefined domains 

In [10]:
domains.domains

{'Tropical_Pacific': [140, 220, -25, 25],
 'SW_Pacific': [172.5, 190, -22.5, -12],
 'Fiji': [175, 183, -21, -15],
 'NZ': [161, 181, -50, -30],
 'Pacific': [140, 240, -50, 25],
 'C3S_download': [100, 240, -50, 30],
 'Water_Watch': [120, 240, -38, 22.5]}

### provider (always CDS for now)

In [11]:
provider = 'CDS'

### variable name

In [12]:
var_name = 'tprate'

### list of GCMs 

In [13]:
list_GCMs = ['ECMWF','UKMO','METEO_FRANCE','CMCC','DWD', 'NCEP', 'JMA', 'ECCC']

### lag in months (if need to download older forecasts)

In [18]:
lag = 0 

### get today's date 

In [19]:
date = dateparser.parse('today')

In [20]:
date

datetime.datetime(2021, 10, 19, 13, 32, 32, 906478)

### apply lag 

In [21]:
date = date - relativedelta(months=lag)

In [22]:
print(f"will download the forecasts issued in {date:%B %Y}")

will download the forecasts issued in October 2021


### path to download the realtime forecasts 

In [17]:
gcm_path = pathlib.Path(f'/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts')

In [18]:
if not(gcm_path.exists()): 
    gcm_path.mkdir(parents=True)

### use dask to parallelize the download of the forecasts 

In [19]:
list_GCMs

['ECMWF', 'UKMO', 'METEO_FRANCE', 'CMCC', 'DWD', 'NCEP', 'JMA', 'ECCC']

In [20]:
downloaded_files = []

for GCM in list_GCMs:
    
    args = dict(GCM=GCM,
    varname=var_name,
    year=date.year,
    month=date.month,
    leadtimes=[1, 2, 3, 4, 5, 6],
    opath=gcm_path.joinpath(GCM).joinpath(var_name.upper()),
    domain=domains.domains['C3S_download'],
    file_format='netcdf',
    level='surface',
    max_retry=3)
    
    downloaded_files.append(delayed(C3S.download)(**args))


In [21]:
with ProgressBar():
    downloaded_files = compute(downloaded_files)

[                                        ] | 0% Completed |  0.0s
attempting to download tprate for GCM METEO_FRANCE, for level surface, year 2021, month 10, in netcdf, attempt 1 of 3


attempting to download tprate for GCM UKMO, for level surface, year 2021, month 10, in netcdf, attempt 1 of 3


attempting to download tprate for GCM NCEP, for level surface, year 2021, month 10, in netcdf, attempt 1 of 3


attempting to download tprate for GCM JMA, for level surface, year 2021, month 10, in netcdf, attempt 1 of 3


attempting to download tprate for GCM DWD, for level surface, year 2021, month 10, in netcdf, attempt 1 of 3


attempting to download tprate for GCM ECCC, for level surface, year 2021, month 10, in netcdf, attempt 1 of 3


attempting to download tprate for GCM CMCC, for level surface, year 2021, month 10, in netcdf, attempt 1 of 3


attempting to download tprate for GCM ECMWF, for level surface, year 2021, month 10, in netcdf, attempt 1 of 3

[                               

2021-10-15 13:20:43,439 INFO Welcome to the CDS
2021-10-15 13:20:43,439 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2021-10-15 13:20:43,441 INFO Welcome to the CDS
2021-10-15 13:20:43,442 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2021-10-15 13:20:43,445 INFO Welcome to the CDS
2021-10-15 13:20:43,446 INFO Welcome to the CDS
2021-10-15 13:20:43,448 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2021-10-15 13:20:43,447 INFO Welcome to the CDS
2021-10-15 13:20:43,447 INFO Welcome to the CDS
2021-10-15 13:20:43,450 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2021-10-15 13:20:43,447 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2021-10-15 13:20:43,446 INFO Welcome to the CDS
2021-10-15 13:

[                                        ] | 0% Completed |  1.6s

2021-10-15 13:20:43,747 INFO Request is queued
2021-10-15 13:20:43,751 INFO Request is queued
2021-10-15 13:20:43,754 INFO Request is queued
2021-10-15 13:20:43,757 INFO Request is queued
2021-10-15 13:20:43,764 INFO Request is queued
2021-10-15 13:20:43,777 INFO Request is queued
2021-10-15 13:20:43,782 INFO Request is queued
2021-10-15 13:20:43,784 INFO Request is queued


[                                        ] | 0% Completed | 11.0s

...

[###################################     ] | 87% Completed |  2min  6.5s

 99%|█████████▉| 5.42M/5.45M [00:06<00:00, 535kB/s]

[###################################     ] | 87% Completed |  2min  6.6s

2021-10-15 13:22:48,738 INFO Download rate 660.2K/s


[###################################     ] | 87% Completed |  2min  6.9s
/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/CMCC/TPRATE/ensemble_seas_forecasts_tprate_from_2021_10_CMCC.netcdf downloaded OK

[########################################] | 100% Completed |  2min  7.0s


In [22]:
# for GCM in list_GCMs: 
    
#     f = C3S.download(
#     GCM=GCM,
#     varname=var_name,
#     year=date.year,
#     month=date.month,
#     leadtimes=[1, 2, 3, 4, 5, 6],
#     opath=gcm_path.joinpath(GCM).joinpath(var_name.upper()),
#     domain=domains.domains['C3S_download'],
#     file_format='netcdf',
#     level='surface',
#     max_retry=3,
#     )   
#     lfiles.append(f)

### check dimensions and return the total number of members in the C3S MME 

In [23]:
import xarray as xr

In [24]:
members = []
for f in downloaded_files[0]: 
    dset = xr.open_dataset(f)
    members.append(dset.dims['number'])
    print(dset.dims)
    if 'system' in dset.dims:
        print(dset['system'])
    

Frozen({'longitude': 141, 'latitude': 81, 'number': 51, 'time': 5})
Frozen({'longitude': 141, 'latitude': 81, 'number': 60, 'time': 5})
Frozen({'longitude': 141, 'latitude': 81, 'number': 51, 'time': 5})
Frozen({'longitude': 141, 'latitude': 81, 'number': 50, 'time': 5})
Frozen({'longitude': 141, 'latitude': 81, 'number': 50, 'time': 5})
Frozen({'longitude': 141, 'latitude': 81, 'number': 120, 'time': 5})
Frozen({'longitude': 57, 'latitude': 33, 'number': 78, 'time': 5})
Frozen({'longitude': 141, 'latitude': 81, 'number': 10, 'time': 5, 'system': 2})
<xarray.DataArray 'system' (system: 2)>
array([1, 2], dtype=int32)
Coordinates:
  * system   (system) int32 1 2
Attributes:
    long_name:  system


In [25]:
import numpy as np 

In [26]:
np.array(members).sum()

470